In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from sklearn.model_selection import train_test_split
import random
import math

In [ ]:
#!ls drive/MyDrive/ColabNotebooks/DSCI471/Project/Project/data/resized/resized

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip3 install opencv-python

In [ ]:
img_dir="drive/MyDrive/ColabNotebooks/DSCI471/Project/Project/data/resized/resized/"
imgs=os.listdir(img_dir)

FileNotFoundError: ignored

In [ ]:
img=cv2.imread(img_dir+imgs[0])

In [ ]:
img.shape

In [ ]:
img2=cv2.imread(img_dir+imgs[0])
print(img2.shape)
plt.imshow(img2)
plt.show()

In [ ]:
y_max=float('-inf')
x_max=float('-inf')
i_x=-1
i_y=-1
for i in range(len(imgs)):
  img=cv2.imread(img_dir+imgs[i])
  y_ax=img.shape[0]
  x_ax=img.shape[1]
  if y_ax>y_max:
    y_max=y_ax
    i_y=i
  if x_ax>x_max:
    x_max=x_ax
    i_x=i

In [ ]:
print(y_max, i_y)
print(x_max, i_x)

In [ ]:
y_max=2296
i_y=1085
x_max=2458
i_x=6631

In [ ]:
print(img_dir+imgs[i_y])

In [ ]:
imgy=cv2.imread(img_dir+imgs[i_y])/255
imgy=cv2.cvtColor(imgy, cv2.COLOR_BGR2HSV)
print(imgy.shape)
plt.imshow(imgy)
plt.show()

In [ ]:
imgx=cv2.imread(img_dir+imgs[i_x])
print(imgx.shape)
plt.imshow(imgx)
plt.show()

In [ ]:
x=(1, 2, 3)
print(x[0])

1


In [ ]:
for i, ID in enumerate(imgs):
  print(i, ID)

In [ ]:
im=np.array(tf.image.resize_with_pad(img, y_max, x_max, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR,
                   antialias=False))
plt.imshow(im)
plt.show()

NameError: ignored

In [ ]:
type(np.array(im))

numpy.ndarray

In [ ]:
class CustomGenerator(tf.keras.utils.Sequence):
  def __init__(self, image_files, label_map, batch_size=20, dim=(y_max, x_max, 3), n_classes=50,
               shuffle=True):
    self.image_files=image_files
    self.label_map=label_map
    self.image_dir="drive/MyDrive/ColabNotebooks/DSCI471/Project/Project/data/resized/resized/"
    self.batch_size=batch_size
    self.dim=dim
    self.n_classes=n_classes
    self.shuffle=shuffle
    self.on_epoch_end()

  def __len__(self):
    return int(np.floor(len(self.image_files)/self.batch_size))
  
  def __getitem__(self, index):
    indexes=self.indexes[index*self.batch_size:(index+1)*self.batch_size]
    temp_IDs=[self.image_files[k] for k in indexes]
    X, y = self.__data_generation(temp_IDs)
    return X, y
  
  def __data_generation(self, temp_IDs):
    X=np.empty((self.batch_size, *self.dim))
    y=np.empty((self.batch_size), dtype=int)

    c=0
    for ID in temp_IDs:
      if ID.split('_')[1]=='Du╠êrer':
        print(ID.split('_'))
        continue
      X[c,]=self.__data_preprocess(ID)
      plt.imshow(X[c,])
      plt.show()
      if ID.split('_')[0]=='Albrecht':
        y[c]=self.label_map['Albrecht_Durer']
        print('here 2')
      else:
        y[c]=self.label_map[ID.rpartition('_')[0]]
      c+=1

    return X, tf.keras.utils.to_categorical(y, num_classes=self.n_classes)

  def __data_preprocess(self, temp_ID):
    #DATA PREPROCESSING HERE
    print(self.image_dir+temp_ID)
    img=cv2.imread(self.image_dir+temp_ID)
    img=img/255
    img=np.array(tf.image.resize_with_pad(img, self.dim[0], self.dim[1], method=tf.image.ResizeMethod.BILINEAR,
                   antialias=False))
    plt.imshow(img)
    plt.show
    return img

  def on_epoch_end(self):
    self.indexes=np.arange(len(self.image_files))
    if self.shuffle==True:
      np.random.shuffle(self.indexes)

In [ ]:
artists=pd.read_csv('drive/MyDrive/ColabNotebooks/DSCI471/Project/Project/data/artists.csv')

In [ ]:
artists_arr=artists.name.unique()
print(artists_arr)

['Amedeo Modigliani' 'Vasiliy Kandinskiy' 'Diego Rivera' 'Claude Monet'
 'Rene Magritte' 'Salvador Dali' 'Edouard Manet' 'Andrei Rublev'
 'Vincent van Gogh' 'Gustav Klimt' 'Hieronymus Bosch' 'Kazimir Malevich'
 'Mikhail Vrubel' 'Pablo Picasso' 'Peter Paul Rubens'
 'Pierre-Auguste Renoir' 'Francisco Goya' 'Frida Kahlo' 'El Greco'
 'Albrecht Dürer' 'Alfred Sisley' 'Pieter Bruegel' 'Marc Chagall'
 'Giotto di Bondone' 'Sandro Botticelli' 'Caravaggio' 'Leonardo da Vinci'
 'Diego Velazquez' 'Henri Matisse' 'Jan van Eyck' 'Edgar Degas'
 'Rembrandt' 'Titian' 'Henri de Toulouse-Lautrec' 'Gustave Courbet'
 'Camille Pissarro' 'William Turner' 'Edvard Munch' 'Paul Cezanne'
 'Eugene Delacroix' 'Henri Rousseau' 'Georges Seurat' 'Paul Klee'
 'Piet Mondrian' 'Joan Miro' 'Andy Warhol' 'Paul Gauguin' 'Raphael'
 'Michelangelo' 'Jackson Pollock']


In [ ]:
artists_map={}
for i in range(len(artists_arr)):
  art=artists_arr[i]
  if art.split(' ')[0]=='Albrecht':
    artists_map['Albrecht_Durer']=i
  else:
    art='_'.join(art.split(' '))
    artists_map[art]=i
print(artists_map)

{'Amedeo_Modigliani': 0, 'Vasiliy_Kandinskiy': 1, 'Diego_Rivera': 2, 'Claude_Monet': 3, 'Rene_Magritte': 4, 'Salvador_Dali': 5, 'Edouard_Manet': 6, 'Andrei_Rublev': 7, 'Vincent_van_Gogh': 8, 'Gustav_Klimt': 9, 'Hieronymus_Bosch': 10, 'Kazimir_Malevich': 11, 'Mikhail_Vrubel': 12, 'Pablo_Picasso': 13, 'Peter_Paul_Rubens': 14, 'Pierre-Auguste_Renoir': 15, 'Francisco_Goya': 16, 'Frida_Kahlo': 17, 'El_Greco': 18, 'Albrecht_Durer': 19, 'Alfred_Sisley': 20, 'Pieter_Bruegel': 21, 'Marc_Chagall': 22, 'Giotto_di_Bondone': 23, 'Sandro_Botticelli': 24, 'Caravaggio': 25, 'Leonardo_da_Vinci': 26, 'Diego_Velazquez': 27, 'Henri_Matisse': 28, 'Jan_van_Eyck': 29, 'Edgar_Degas': 30, 'Rembrandt': 31, 'Titian': 32, 'Henri_de_Toulouse-Lautrec': 33, 'Gustave_Courbet': 34, 'Camille_Pissarro': 35, 'William_Turner': 36, 'Edvard_Munch': 37, 'Paul_Cezanne': 38, 'Eugene_Delacroix': 39, 'Henri_Rousseau': 40, 'Georges_Seurat': 41, 'Paul_Klee': 42, 'Piet_Mondrian': 43, 'Joan_Miro': 44, 'Andy_Warhol': 45, 'Paul_Gaugui

In [ ]:
for i in range(len(imgs)):
  img_try=imgs[i].rpartition('_')[0]
  if img_try not in artists_map:
    print(img_try, i)

In [ ]:
n_classes=len(artists.name.unique())

In [ ]:
traingen=CustomGenerator(imgs, artists_map)

In [ ]:
X, y=traingen[0]
print(X)
print(y)

In [ ]:
print(y[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [ ]:
for i in range(len(X)):
  plt.imshow(X[i])
  plt.show()

In [ ]:
train_imgs=random.sample(imgs, math.floor(0.8*len(imgs)))


In [ ]:
model = keras.Sequential([
    layers.Conv2D(32, (5,5), input_shape=(y_max, x_max, 3)),
    layers.MaxPool2D(pool_size=2, strides=2),
    layers.Conv2D(64, (5,5), activation='relu', padding='same'),
    layers.MaxPool2D(pool_size=2, strides=2),
    layers.Flatten(),
    layers.Dense(128, activation='tanh'),
    layers.Dense(n_classes, activation='softmax')
])